In [ ]:
mutable struct File
  name::String
  size::Int64
  File(name::String, size::Int64) = new(name, size)
end

In [ ]:
mutable struct Directory
  name::String
  subDirectories::Vector{Directory}
  files::Vector{File}
  parent::Union{Directory, Nothing}
  size::Int64
  Directory(name::String, parent::Union{Directory, Nothing}) = new(name, Vector{Directory}(), Vector{File}(), parent)
end


In [ ]:
size(file::File) =
  begin
    return file.size
  end
size(dir::Directory) =
  begin
    return dir.size
  end
calcSubTreeSize(dir::Directory) =
  begin
    map(calcSubTreeSize, dir.subDirectories)
    filesSize = sum(map(size, dir.files))
    subDirectoriesSize = sum(map(size, dir.subDirectories))
    dir.size = filesSize + subDirectoriesSize
  end
subDirsOfSizeBetween(dir::Directory, a::Int64, b::Int64, currFound::Vector{Int64}) =
  begin
    map(x -> subDirsOfSizeBetween(x, a, b, currFound), dir.subDirectories)
    if Base.between(dir.size, a, b)
      push!(currFound, dir.size)
    end
  end

In [ ]:
root = Directory("/", nothing)
wd = root
for line in eachline("input.txt")
  sections = map(x -> String(x), split(line))
  if (sections[1] == "\$")
    if (sections[2] == "cd")
      if (sections[3] == "/")
        while (wd.name != "/")
          wd = wd.parent
        end
      elseif (sections[3] == "..")
        wd = wd.parent
      else
        dirToSwitchPos = findfirst(dir -> dir.name == sections[3], wd.subDirectories)
        wd = wd.subDirectories[dirToSwitchPos]
      end
    end

  elseif (sections[1] == "dir")
    push!(wd.subDirectories, Directory(sections[2], wd))

  else
    push!(wd.files, File(sections[2], parse(Int64, sections[1])))
  end
end
calcSubTreeSize(root)

In [ ]:
# task 1
mediumDirSizes = Vector{Int64}()
subDirsOfSizeBetween(root, 0, 100000, mediumDirSizes)
sum(mediumDirSizes)

In [ ]:
# task 2
fsMaxSize = 70000000
freeSizeNeeded = 30000000
neededToDelete = freeSizeNeeded - (fsMaxSize - root.size)
sizesForPossibleDeletion = Vector{Int64}()
subDirsOfSizeBetween(root, neededToDelete, freeSizeNeeded, sizesForPossibleDeletion)
minimum(sizesForPossibleDeletion)